In [1]:
import scipy.sparse as spr
import scipy.sparse.linalg as spla
import sys
path2oti = '../../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.real   as r
import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 

# import pyoti.static.onumm1n1    as dual
# import pyoti.static.mdnum2      as md2
# import pyoti.static.mdnum3      as md3
# import pyoti.static.mdnum5      as md5
# import pyoti.static.mdnum6      as md6
# import pyoti.static.mdnum10     as md10
# import pyoti.static.onumm1n10   as om1n10
# import pyoti.static.onumm1n2    as om1n2
# import pyoti.static.onumm1n5    as om1n5
# import pyoti.static.onumm5n5    as om5n5
# import pyoti.static.onumm2n2    as om2n2
# import pyoti.static.onumm3n3    as om3n3
# import pyoti.static.onumm2n10    as om2n10




%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

e  = oti.e
np = oti.np

import pyvista as pv
# p = pv.BackgroundPlotter()
pv.set_plot_theme('document')

global times
from timeit import default_timer as time

from matplotlib import rc

## for Palatino and other serif fonts use:
rc('font',**{'family':'serif','serif':['Palatino'], 'size':16})
rc('text', usetex=True)

In [2]:
Th = fem.mesh.load('mesh.msh')


A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.


In [3]:
Th2 = fem.mesh.load('mesh.msh')

In [4]:
Th.x[5,0]

0.0000

In [5]:
# perturb
order=10
for i in range(Th.nnodes):
    
    x = Th.x[i,0].copy()
    y = Th.y[i,0].copy()

    r = oti.sqrt(x**2+y**2).real
    if x.real ==0 and y.real == 0:
        theta = 0
    else:
        theta = np.arctan2(y.real,x.real)
    # end if 

    if r < 0.3:
        Th.x[i,0] = r*oti.cos(theta +oti.e(1,order=order) )+(0.3-r)/0.3*oti.e(2,order=order)
        Th.y[i,0] = r*oti.sin(theta +oti.e(1,order=order) )#+(0.3-r)*oti.e(2,order=order)
    elif r < 0.6:
        hx = (0.6-r)*oti.cos(theta +oti.e(1,order=order) )
        hx = hx-hx.real
        hy = (0.6-r)*oti.sin(theta +oti.e(1,order=order) )
        hy = hy-hy.real

        Th.x[i,0] = x.real + hx
        Th.y[i,0] = y.real + hy
            

In [6]:
Th.x[397,0]

-0.1000 - 0.0000 * e([1]) + 0.6667 * e([2]) + 0.0500 * e([[1,2]]) + 0.0000 * e([[1,3]]) - 0.0042 * e([[1,4]]) - 0.0000 * e([[1,5]]) + 0.0001 * e([[1,6]]) + 0.0000 * e([[1,7]]) - 0.0000 * e([[1,8]]) - 0.0000 * e([[1,9]]) + 0.0000 * e([[1,10]])

In [7]:
Th.group_names['Crack']

{'dim': 0,
 'id': 6,
 'nodes': array([5], dtype=uint64),
 'entities': array([6], dtype=int32),
 'members': [{'types': array([15], dtype=int32),
   'tags': [array([2], dtype=uint64)],
   'nodes': array([6], dtype=uint64),
   'indices': [array([[5]], dtype=uint64)]}]}

In [8]:
import pyvista as pv
p = pv.BackgroundPlotter()
args_cbar = dict(height=0.75, vertical=True, position_x=0.05, 
                 position_y=0.05, interactive=False,
                 title_font_size=30, label_font_size=30)

In [9]:
Th2.x = Th.x.taylor_integrate([1,2],[0.5,0.2])
Th2.y = Th.y.taylor_integrate([1,2],[0.5,0.2])

# grid = Th2.to_pv(pd = [np.sqrt(Th.x.get_deriv(2)**2+Th.y.get_deriv(2)**2).real],pd_names=['x-pert'])
grid = Th2.to_pv(dims=1)

p.clear()

p.add_mesh(grid, show_edges=True, line_width=2, categories=16, cmap='jet', scalar_bar_args=args_cbar)

p.show()